In [ ]:
!pip install msal msal_extensions
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Lin

In [ ]:

# onedrive 관련
from io import StringIO
from msal import PublicClientApplication
import requests
import os
from msal_extensions import FilePersistence, PersistedTokenCache
import pandas as pd

# 모델 관련
import torch
from google.colab import drive
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
from datasets import load_dataset, Dataset

In [ ]:
def get_ms_token(client_id, authority, scopes):
    # 캐시 파일 경로 설정
    cache_file_path = os.path.expanduser('~/.msal_cache.json')

    # MSAL Extensions를 사용하여 파일 기반 캐시 생성
    persistence = FilePersistence(cache_file_path)
    token_cache = PersistedTokenCache(persistence)

    # MSAL 앱 생성
    app = PublicClientApplication(client_id, authority=authority, token_cache=token_cache)

    # 캐시에서 기존 계정 확인
    accounts = app.get_accounts()
    if accounts:
        # 첫 번째 계정 선택 (여러 계정이 있을 경우 적절히 선택)
        result = app.acquire_token_silent(scopes, account=accounts[0])
        if 'access_token' in result:
            print('캐시된 토큰을 사용합니다.')
        else:
            print('캐시에서 유효한 토큰을 찾을 수 없습니다. 인증을 진행합니다...')
    else:
        # 디바이스 코드 플로우를 통한 새 인증 진행
        flow = app.initiate_device_flow(scopes=scopes)
        if 'user_code' not in flow:
            raise ValueError('디바이스 플로우 생성에 실패했습니다. 설정을 확인하세요.')
        print(f"다음 URL로 이동하여 코드를 입력하세요: {flow['verification_uri']}")
        print(f"인증 코드: {flow['user_code']}")
        result = app.acquire_token_by_device_flow(flow)

    if 'access_token' in result:
        print('인증에 성공했습니다!')
        headers = {'Authorization': f"Bearer {result['access_token']}"}
        return headers
    else:
        print('인증에 실패했습니다.')
        return None

In [ ]:
def list_onedrive_files(headers, target_name, folder_id=None):
    """
    OneDrive 폴더의 파일 목록을 가져옵니다.

    Parameters:
        headers (str): MSAL을 통해 얻은 인증 토큰
        target_name(str): 내가 찾고자 하는 파일의 이름
        folder_id (str): 폴더의 ID (None일 경우 루트 폴더)

    Returns:
        dict: 파일 이름과 파일 ID의 매핑
    """
    base_url = "https://graph.microsoft.com/v1.0/me/drive"
    url = f"{base_url}/items/{folder_id}/children" if folder_id else f"{base_url}/root/children"

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        folders = response.json().get('value', [])
        # 확장자가 없는 폴더만 사전에 추가
        folder_mapping = {folder['name']: folder['id'] for folder in folders}
        print(folder_mapping)
        for name in folder_mapping:
            if name == target_name:
                return folder_mapping[name]
            elif name.find('.') == -1:
                result = list_onedrive_files(headers, target_name, folder_mapping[name])
                if result:
                    return result

    elif response.status_code == 404:
        print(f"폴더를 찾을 수 없습니다: {folder_id}")
        return False

    else:
        print(f"폴더 파일 목록을 가져오는 데 실패했습니다: {response.status_code} - {response.text}")
        return False

In [ ]:
def load_csv_from_onedrive(headers, file_id) -> pd.DataFrame:
    url = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}/content"
    response = requests.get(url, headers=headers, stream=True)

    if response.status_code == 200:
        csv = pd.read_csv(StringIO(response.text))
        print(f"파일 탐색 성공")
        return csv
    else:
        print(f"파일 다운로드 실패: {response.status_code} - {response.text}")

In [ ]:
# Azure 앱 정보
GRAPH_API_URL = 'https://graph.microsoft.com/v1.0'
CLIENT_ID = 'ef053b61-d7f1-4942-97d4-bb79fa475a01'  # 앱 등록에서 가져온 클라이언트 ID
AUTHORITY = 'https://login.microsoftonline.com/f09a4ef3-978d-434e-89da-a29b9f9f3c32'  # 테넌트 ID 또는 'common'
SCOPES = ['Files.ReadWrite.All']  # 필요 권한 설정

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
headers = get_ms_token(CLIENT_ID, AUTHORITY, SCOPES)

file_id = list_onedrive_files(headers, 'train.csv')

다음 URL로 이동하여 코드를 입력하세요: https://microsoft.com/devicelogin
인증 코드: IQDGBGNV8
인증에 성공했습니다!
{'첨부 파일': '01UUMNEVPWL73GRXSYZNDZUQO7WAMFJK5U', 'AI_Bio_Research': '01UUMNEVLFHOTOGTSKKBEI7SGXFGRI3OYV', 'dacon': '01UUMNEVP3V7HTIIDC4ND2OGQA2MDF5DEB'}
{}
{'1000_이론 및 실습': '01UUMNEVIOEEJ5BSHJDRDID2TYJ5XBI5FL', '2000_논문 리뷰': '01UUMNEVM6QJ2EX34MGJCK3CW4P3ZLTLRY', '3000_연구 수행': '01UUMNEVKW4UAVQVOYXZFLLX7OBYGDEPD5', '5000_코드 및 데이터 관리': '01UUMNEVPPHAEXKXCXLRC2BJGY34Y4LP3Y', '6000_블로그 관리': '01UUMNEVJOXNVK6OM3DFB3IHVUQCCRV5OS', '7000_컨퍼런스 및 네트워킹': '01UUMNEVIGWDP3RQ2VKNDIU56S6IRD7GMY', '8000_참고 자료': '01UUMNEVMVOBJCEVTAFRFIFBKXT4H5BAE4', '신약_AI_개발자로_거듭나기_5개년_계획.xlsx': '01UUMNEVOOB3TVWMZNNZBLLSBKTADL3TQ7'}
{}
{'2100_신약 개발 AI': '01UUMNEVISQIWEGPWPN5BIDUDQCORZHVMT', '2200_단백질 - 화합물 결합 예측': '01UUMNEVPLSHJGBXWHJBC2QHUXCWFMBHRA', '2300 MD Simulation': '01UUMNEVO7ASKOUGXEWBFZLUDMCUOMBK6H', '2900_기타 논문': '01UUMNEVLZS4GQQK6KEZDLVCJLMCOYOEP3'}
{'논문_A.md': '01UUMNEVOXMVZUKCUFMNG3NLIQV7UHNGI3', '논문_B.md': '01UUMNEVPYXDZ5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 사전 학습된 모델과 토크나이저 로드
model_name = 'gogamza/kobart-base-v2'   # koBART 모델 사용
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

In [ ]:
# 데이터셋 로드
dataset = Dataset.from_pandas(load_csv_from_onedrive(headers, file_id)).train_test_split(test_size=0.1)
dataset

파일 탐색 성공


DatasetDict({
    train: Dataset({
        features: ['ID', 'input', 'output'],
        num_rows: 145136
    })
    test: Dataset({
        features: ['ID', 'input', 'output'],
        num_rows: 16127
    })
})

In [ ]:
# 모델 성능 테스트를 위한 소량 데이터 구축
# small_dataset = dataset.select(range(int(len(dataset) * 0.1))).train_test_split(test_size=0.1)
# small_dataset

In [ ]:
# 데이터 전처리 함수
def preprocess_function(examples):
    inputs = [ex for ex in examples['input']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['output'], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def preprocess_function(examples):
    # input과 output이 문자열인지 확인하고, None이면 빈 문자열로 대체
    inputs = [ex if isinstance(ex, str) else "" for ex in examples['input']]
    outputs = [ex if isinstance(ex, str) else "" for ex in examples['output']]

    # 입력 데이터 토크나이징
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # 출력 데이터 토크나이징 (text_target 사용)
    labels = tokenizer(text_target=outputs, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# 데이터셋 전처리
# tokenized_datasets = small_dataset.map(preprocess_function, batched=True)
tokenized_datasets = dataset.map(preprocess_function, batched=True)

tokenized_datasets

Map:   0%|          | 0/145136 [00:00<?, ? examples/s]

Map:   0%|          | 0/16127 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'input', 'output', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 145136
    })
    test: Dataset({
        features: ['ID', 'input', 'output', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 16127
    })
})

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./content/drive/MyDrive/Colab",
    evaluation_strategy="epoch",  # 스텝마다 검증
    eval_steps=1000,  # 1000스텝마다 검증
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # 배치 크기 증가
    gradient_accumulation_steps=2,  # 그래디언트 누적
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,  # 에포크 수 증가
    predict_with_generate=True,
    fp16=True,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to=[],
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    warmup_steps=500,  # Warmup 스텝 설정
    lr_scheduler_type="cosine",  # Cosine 스케줄러 사용
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # 조기 종료
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-4877a3d8ff0e>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# 모델 학습
checkpoint_dir = "/contents/drive/MyDrive/Colab/checkpoint-27211"  # 최신 체크포인트 경로

# dir 존재가 아니라 실제 파일이 존재하는지 확인하는 로직으로 변경해야 함
if os.path.exists(checkpoint_dir) and os.listdir(checkpoint_dir):  # 파일이 존재하는지 확인
    print("체크포인트를 로드합니다.")
    trainer.train(resume_from_checkpoint=True)
else:
    print("새로운 훈련을 시작합니다.")
    trainer.train()

새로운 훈련을 시작합니다.


Epoch,Training Loss,Validation Loss
1,0.093800,0.078750
2,0.064300,0.062790
3,0.047900,0.056412
4,0.039100,0.053411


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


.pt 파일을 onedrive에 저장해놓고 뽑아쓰기

In [ ]:
# 모델 평가
trainer.evaluate()

In [ ]:
# 테스트 데이터로 예측
def generate_review(text):
    # inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length")

    inputs = tokenizer(text, return_tensors="pt")  # Tokenize the input text
    inputs = {k: v.to(model.device) for k, v in inputs.items()} # Move inputs to the same device as the model
    outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# 예시 입력
test_input = "핸외옇헹 중 빵문했뎐 쑥뱍 씨썩륀냐 움식젊 휵뀌룰 낢뀔 떼, 혹끊 했외 쩨룹 쭝 욋쿡인 찐궂 몲룩케 긁룰 숙교 십을 떼 유용한 빻펍위 있닥. 직규샹 얼턴 변엌깃돛 윌끌 쑤 업눈 윌멍 ‘옜엄뷔엔피 쩨’댜. 예여삐옌빗 슉쏘 추윈 몲랭 한꾸귄많 얄랐쩨또록 휴뀔룰 냠퀸 텟섣 퓔룟됐따. 몸움과 쟈욺읠 따앙한 좋함쁠로 어려 켱욺위 쑤를 만듦려네눈 빵쉬끼댜."
predicted_output = generate_review(test_input)
print("Predicted Output:", predicted_output)

In [ ]:
# 모델 저장
model.save_pretrained("./contents/drive/MyDrive/colab")
tokenizer.save_pretrained("./contents/drive/MyDrive/colab")

pt 파일 저장해놓고, model을 불러온 뒤 onedrive에서 test.csv 가져와서 input 넣고 돌리면 된다.